In [1]:
from gensim.models import FastText
import pandas as pd

### Train fasttext embedding

In [2]:
etcsl = pd.read_csv("etcsl.csv")
etcsri = pd.read_csv("etcsri.csv")

/var/folders/h6/_dg2f8_54bv4lnnn36ny5ggw0000gn/T/ipykernel_62336/3067424915.py:2: DtypeWarning: Columns (8,15,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  etcsri = pd.read_csv("etcsri.csv")


In [3]:
lines = []
last_seen_id_line = 100000
temp = []
for idx, row in etcsl.iterrows():
    if last_seen_id_line != row.id_line:
        if temp != []:
            lines.append(temp)
        last_seen_id_line = row.id_line
        temp = []
        temp.append(row.form)
    else:
        temp.append(row.form)


In [4]:
etcsri_lines = []
last_seen_line = 100000
last_seen_page = 100000
temp = []
for idx, row in etcsri.iterrows():
    page, line, word_id = tuple(row.id_word.split("."))
    if last_seen_line != line or last_seen_page!= page:
        if temp != []:
            etcsri_lines.append(temp)
        last_seen_page = page
        last_seen_line = line
        temp = []
        temp.append(row.form)
    else:
        temp.append(row.form)

In [6]:
sux_corpus = lines + etcsri_lines
print("Sumerian sentence in training: {}".format(len(sux_corpus)))

Sumerian sentence in training: 277247


#### Set up FastText Model

In [7]:
model = FastText(vector_size=512, window=10, min_count=2)
model.build_vocab(corpus_iterable=sux_corpus)

In [8]:
model.train(corpus_iterable=sux_corpus, total_examples=len(sux_corpus), epochs=50)  # Adjust epochs as needed

(26995700, 33811600)

#### Test model on some words

In [28]:
similar_words = model.wv.most_similar('kalag-ga', topn=10) ### kalag-ga means strong
similar_words

[('kalag-kalag-ga', 0.9821982979774475),
 ('lag-ga', 0.8817840218544006),
 ('kalag-ga-a', 0.8638339042663574),
 ('kalag-ga-ar', 0.8539075255393982),
 ('kalag-kalag', 0.8514988422393799),
 ('kalag-ga-kam', 0.845146894454956),
 ('zalag-ga', 0.8418394923210144),
 ('ag-ga', 0.8318354487419128),
 ('kalag-ga-ŋa₂', 0.8310603499412537),
 ('nam-kalag-ga', 0.8265538215637207)]

In [29]:
etcsl[etcsl["form"] == "kalag-kalag-ga"]["gw"] ### kalag-kalag-ga means strong

27948     strong
147433    strong
Name: gw, dtype: object

In [31]:
# model.save("sumerian_fasttext.model")